In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
import string

import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%matplotlib widget
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
startyear = 2003
endyear   = 2008

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)
axex = [245.8,251,-75.2,-74.4]
axex2 = [245.7,246.1,-75.11,-74.85]
axex3 = [246.8,247,-74.6,-74.3]
axex4 = [248.93,249.07,-75.35,-74.7]


cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.rain'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

x0,x1,y0,y1 = 3400,3800,7700,8100

In [ ]:
vmax = 100
vmin = -10
linthresh=1
linscale=.2

vmax = 100
vmin = -10
linthresh = .3
linscale = .25


fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#print(fracpos)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,.9,nneg))
colors2 = plt.cm.inferno(np.linspace(0.1, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)


In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
    mask = xr.where(ds.mask==1,2,ds.mask)
    ds = add_lonlat(ds)
    
    ax.set_facecolor(".7")
    
    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    #ax.contour(ds.lon,ds.lat,np.where(ds.mask==3,ds.thickness-ds.surface,np.nan),[450],colors='k',linewidths=.5,transform=ccrs.PlateCarree(),zorder=10)
    
    ax.set_extent(axex,crs=ccrs.PlateCarree())

In [ ]:
def plotmelt(ax,lon,lat,melt,title):
    makebackground(ax,axex)
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,melt,cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),shading='nearest',transform=ccrs.PlateCarree())

    ax.set_title(title,loc='left',fontsize=10)
    
    return im

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.01,right=.98,top=.95,bottom=.2,wspace=.05,hspace=.0)

fig = plt.figure(figsize=(7,4))

#1D forcing
ax = fig.add_subplot(131,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))

ds1 = xr.open_dataset(f'../../results/CrossDots_0.5_tanh_Tdeep0.4_ztcl-500_050.nc')

im = plotmelt(ax,ds1.lon,ds1.lat,np.where(ds1.tmask,ds1.melt,np.nan),'a) 1D forcing')

#3D forcing
ax = fig.add_subplot(132,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))

ds2 = xr.open_dataset(f'../../results/spinup/CrossDots_0.5_mitgcm_{startyear}_{endyear}_100.nc')

im = plotmelt(ax,ds2.lon,ds2.lat,np.where(ds2.tmask,ds2.melt,np.nan),'b) 3D forcing')

#Diff
ax = fig.add_subplot(133,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))

makebackground(ax,axex)
ax.set_extent(axex,crs=ccrs.PlateCarree())
ax.set_title('c) Difference',loc='left',fontsize=10)

#im2 = ax.pcolormesh(ds1.lon,ds1.lat,np.where(ds1.mask==3,100*(ds2.melt-ds1.melt)/ds1.melt,np.nan),cmap='cmo.balance',shading='nearest',transform=ccrs.PlateCarree(),vmin=-100,vmax=100)
im2 = ax.pcolormesh(ds1.lon,ds1.lat,np.where(ds1.mask==3,ds2.melt-ds1.melt,np.nan),cmap='cmo.balance',shading='nearest',transform=ccrs.PlateCarree(),vmin=-10,vmax=10)

#Colorbars

ax = fig.add_subplot(111)
ax.set_visible(False)
cax = inset_axes(ax,width="32%",height="5%",loc='lower left',bbox_to_anchor=(.68,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
cb = plt.colorbar(im2,cax=cax,orientation='horizontal',extend='both')
#cb.set_label('Relative difference (m$_{3D}$ - m$_{1D}$)/m$_{1D}$ [\%]')
cb.set_label('Melt rate difference m$_{3D}$ - m$_{1D}$ [m yr$^{-1}$]')


cax = inset_axes(ax,width="66%",height="5%",loc='lower left',bbox_to_anchor=(.0,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cb = plt.colorbar(im,cax=cax,orientation='horizontal',extend='both')
cb.set_ticks(ticks)
cb.set_ticklabels(ticks)
cb.set_label('Freezing / Melt rate [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_CD_forcing.png',dpi=450)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(131,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = ax.pcolormesh(ds1.lon,ds1.lat,np.where(ds1.mask==3,ds2.D-ds1.D,np.nan),cmap='cmo.balance',shading='nearest',transform=ccrs.PlateCarree(),vmin=-100,vmax=100)
plt.colorbar(im,orientation='horizontal')

ax = fig.add_subplot(132,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = ax.pcolormesh(ds1.lon,ds1.lat,np.where(ds1.mask==3,(ds2.U**2+ds2.V**2)**.5-(ds1.U**2+ds1.V**2)**.5,np.nan),cmap='cmo.balance',shading='nearest',transform=ccrs.PlateCarree(),vmin=-.1,vmax=.1)
plt.colorbar(im,orientation='horizontal')

ax = fig.add_subplot(133,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = ax.pcolormesh(ds1.lon,ds1.lat,np.where(ds1.mask==3,ds2.T-ds1.T,np.nan),cmap='cmo.balance',shading='nearest',transform=ccrs.PlateCarree(),vmin=-1,vmax=1)
plt.colorbar(im,orientation='horizontal')